In [85]:
import requests
import pandas as pd
from pprint import pprint
import sqlite3
from html import unescape
from bs4 import BeautifulSoup
import re
session = requests.session()
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from collections import Counter
from sklearn.metrics import accuracy_score

*Ищем положительные отзывы и очищаем их от тегов*

In [201]:
url_good = f'https://www.kinopoisk.ru/film/688/reviews/?status=good&ord=date&rnd=1601992343&perpage=50'
req = session.get(url_good, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
good = soup.find_all('span', {'itemprop': 'reviewBody'})

In [202]:
cleanr = re.compile('<.*?>')
cleangood = re.sub(cleanr, '', str(good))

*Ищем отрицательные отзывы и очищаем их от тегов*

In [211]:
url_bad = f'https://www.kinopoisk.ru/film/407636/reviews/?status=bad&ord=date&rnd=1601997467&perpage=50&page=2'
req = session.get(url_bad, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
bad = soup.find_all('span', {'itemprop': 'reviewBody'})

In [215]:
cleanr = re.compile('<.*?>')
cleanbad = re.sub(cleanr, '', str(bad))

*Ищем тестовые отзывы, очищаем их от тегов, кладем в список*

In [204]:
url_test = f'https://www.kinopoisk.ru/film/1143242/reviews/'
req = session.get(url_test, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
test = soup.find_all('span', {'itemprop': 'reviewBody'})

In [205]:
test_sp = str(test).split('<span class="_reachbanner_" itemprop="reviewBody">')
for i in test_sp:
    cleant = re.compile('<.*?>')
    cleantest = re.sub(cleant, '', str(i))
    cleanre = re.compile('[^а-яёА-ЯЁ ]')
    cleantest = re.sub(cleanre, '', str(i))
del test_sp[0]

*Приводим текст к нижнему регистру и токенизируем в функции, а затем применяем ее для положительных и отрицательных комментариев* 

In [206]:
def tokenizing(text):
    text = text.lower()
    lemmas = []
    token = word_tokenize(text)
    for word in token:
        an = morph.parse(word)
        first = an[0]
        lemma = first.normal_form
        lemmas.append(lemma)
    return(lemmas)

In [212]:
def low_freq(text):
    freqlist = Counter()
    for word in text:
        if word.isalpha():
            freqlist[word] += 1
    d = dict(freqlist.most_common())
    for w in d:
        if d.get(w)<=2:
            text.remove(w)

In [213]:
tokbad = tokenizing(cleanbad)
tokgood = tokenizing(cleangood)

In [214]:
low_freq(tokbad)
low_freq(tokgood)

*Делим комментарии на 2 множества*

In [196]:
good_comments = set(tokgood)
bad_comments = set(tokbad)
only_bad = bad_comments.difference(good_comments)
only_good = good_comments.difference(bad_comments)

*Создаем список, в который добавляем информацию о тестовых комментариях: 1 - положительный, 0 - отрицательный*

In [197]:
lis = []

In [198]:
def good_or_bad(comment):
    g = 0
    b = 0 
    x = 0
    for i in comment:
        if i in only_good:
            g+=1
        elif i in only_bad:
            b+=1
    if g>=b:
        x = 1
    else:
        x = 0
    lis.append(x)
    return lis

In [199]:
for i in test_sp:
    good_or_bad(i)

*Сравниваем получившийся список с правильными ответом при помощи accuracy*

In [200]:
correct = [1, 1, 1, 1, 0, 0, 1, 1, 0, 1]
accuracy_score(lis, correct)

0.7

**Как можно улучшить программу:**
*1)Добавить визуализацию (облако) для слов, встречающихся только в положительных или отрицательных комментариях;*
*2)Избавиться от служебных слов*